### modules

In [ ]:
if globals().get('LOADED_LOGGER') == None:
    %run LOGGER.ipynb

In [ ]:
if globals().get('LOADED_ANNOTATIONS') == None:
    %run ANNOTATIONS.ipynb

In [ ]:
if globals().get('LOADED_PATTERN_OBSERVER') == None:
    %run PATTERN_OBSERVER.ipynb

### import protection

In [ ]:
if globals().get('LOADED_CHECKBOXES') == None:
    logging.info('LOADED_CHECKBOXES')
    LOADED_CHECKBOXES=True

### imports

In [ ]:
# widgets
from ipywidgets import Layout,HBox,VBox,Checkbox,Button
from ipywidgets import Color # pretty color for apply button

### begin

In [ ]:
'''
SUBJECT
- DATA_CONTAINER observes CHECKBOXES to populate itself

OBSERVER
- CHECKBOXES observes SETTINGS for options
'''
class CHECKBOXES(ISubject):

    ####################################
    # constructor
    ####################################
    def __init__(self,
        name                  : str         = 'CHECKBOXES',
        options               : OPTIONS     = {},                          # key = name of object, value = unique ID
        default               : bool        = False,                       # default value of checkboxes
        width                 : str         = 'auto',
        ):
        
        self._default                       = default
        self._width                         = width
        self._options                       = self._populate_options(options)
        
        self.name                           = name
        self.widget                         = self._make_widget(self._options)
        
        self.applied_settings               = self._current_settings       # update reference
        self._observers                     = []

    ####################################
    # populate defaults
    ####################################
    def _populate_options(self,options : List[str] = list('abcd')) -> OPTIONS :
        if bool(options):
            return options
        else:
            return {'option ' + x : x*5 for x in options}
        
    ####################################
    # observer pattern
    ####################################
    # subject
    def attach(self,observer : IObserver) -> None :
        logging.info(f'OBSERVER PATTERN : {observer.name} : OBSERVES {self.name}')
        self._observers.append(observer)
        
    def detach(self,observer : IObserver) -> None :
        logging.info(f'OBSERVER PATTERN : {observer.name} : STOPS OBSERVING {self.name}')
        self._observers.remove(observer)
        
    def notify(self,info) -> None :
        logging.info(f'OBSERVER PATTERN : {self.name} : {len(self._observers)} OBSERVERS')
        
        for observer in self._observers:
            logging.info(f'OBSERVER PATTERN : {self.name} : NOTIFIES {observer.name}')
            observer.react(self.name,info)
            
    ####################################
    # button state
    ####################################
    def _button_inactive(self,button : Button) -> None :
        button.style.button_color=Color(None).name
        button.disabled = True
        
    def _button_active(self,button : Button,color : str = 'pink') -> None :
        button.style.button_color=color
        button.disabled = False
        
    def _update_button_state(self) -> None :
        if self.applied_settings == self._current_settings:
            # if nothing changed
            self._button_inactive(self._control_widgets[0])
            self._button_inactive(self._control_widgets[1])
        else:
            # if something changed
            self._button_active(self._control_widgets[0],color = 'lightgreen')
            self._button_active(self._control_widgets[1],color = 'pink')

    ####################################
    # button click
    ####################################
    '''NB - need apply button to trigger a change, if use checkbox change to trigger plotting then clicking "all/none/invert" causes multiple sequential intra plots'''
    def _click_apply(self,change) -> None :
        logging.debug(f'{self.name} : CLICKED {change.description}')
        self.applied_settings=self._current_settings   # update checkpoint
        self._update_button_state()                    # update button state
        self._logging()                                # show internals
        self.notify(self.applied_settings)             # notify observers

    '''reset to last applied values'''
    def _click_cancel(self,change) -> None :
        logging.debug(f'{self.name} : CLICKED {change.description}')
        for widget,value in zip(self._option_widgets,self.applied_settings.values()):
            widget.value = value

    '''uncheck all'''
    def _click_none(self,change) -> None :
        logging.debug(f'{self.name} : CLICKED {change.description}')
        for c in self._option_widgets:
            c.value = False

    '''check all'''
    def _click_all(self,change) -> None :
        logging.info(f'{self.name} : CLICKED {change.description}')
        for c in self._option_widgets:
            c.value = True

    '''flip all'''
    def _click_invert(self,change) -> None :
        logging.debug(f'{self.name} : CLICKED {change.description}')
        for c in self._option_widgets:
            c.value=not c.value

    ####################################
    # widget onchange
    ####################################
    def _onchange_checkbox(self,change) -> None :
        logging.debug(f'{self.name} : CHANGED {change.owner.description} = {change.new}')
        self._update_button_state() # update button state
    
    ####################################
    # build GUI
    ####################################
    def _make_widget(self,options : OPTIONS) -> VBox :
        # controls
        controls = HBox(self._make_controls())
        
        # options
        options = VBox(
            children    = self._make_options(options),
            layout      = Layout(height='300px')
        )
        
        # return
        return VBox(
            children    = [controls,options],
            layout      = Layout(border='1px solid black',width=self._width,overflow_x='scroll')
        )
    
    def _make_controls(self) -> BUTTONS :
        button_apply = Button(
            description = 'apply',
            disabled    = True,
            tooltip     = 'apply changes',
        )
        button_apply.on_click(self._click_apply)

        button_cancel = Button(
            description = 'cancel',
            disabled    = True,
            tooltip     = 'retrieve last applied values',
        )
        button_cancel.on_click(self._click_cancel)
        
        button_all = Button(
            description = 'all',
            tooltip     = 'select all',
        )
        button_all.on_click(self._click_all)
        
        button_none = Button(
            description = 'none',
            tooltip     = 'clear selection',
        )
        button_none.on_click(self._click_none)
        
        button_invert = Button(
            description = 'invert',
            tooltip     = 'invert selection',
        )
        button_invert.on_click(self._click_invert)

        return [
            button_apply,
            button_cancel,
            button_all,
            button_none,
            button_invert,
        ]

    def _make_options(self,options : OPTIONS) -> CHECKBOXES :
        # checkboxes
        widgets=[
            Checkbox(
                description=option,
                layout=Layout(height='15px'),
                indent=False,
                value=self._default
            ) for option in options
        ]
        
        # register checkboxes
        for c in widgets:
            c.observe(self._onchange_checkbox,names='value')

        # return
        return widgets
    
    ####################################
    # properties
    ####################################
    @property
    def _control_widgets(self) -> BUTTONS :
        return self.widget.children[0].children
    
    @property
    def _option_widgets(self) -> CHECKBOXES :
        return self.widget.children[1].children
    
    @_option_widgets.setter
    def _option_widgets(self,new_options : CHECKBOXES) -> None:
        self.widget.children[1].children=new_options
        
    @property
    def _current_settings(self) -> OPTIONID_VALUES :
        return self._option_identifier_vs_option_value(self._option_widgets,self._options)
    
    ####################################
    # functions
    ####################################
    '''
    a=CHECKBOXES({'option '+str(x):str(x)*10 for x in range(0,10)})
    display(a.widget)
    a.update_options({'option '+str(x):str(x)*10 for x in range(5,20)})
    '''
    def update_options(self,new_options : OPTIONS) -> None:
        logging.info(f'{self.name} : update_options')

        ################################################################
        # if existing option exists, replace newly created widget with existing one
        ################################################################
        # unique identifier for _options
        new_options_list      = self._dict2list(new_options)
        old_options_list      = self._dict2list(self._options)

        # create new widgets from recently provided new_options
        new_option_widgets    = self._make_options(new_options)
        new_applied_settings  = self._option_identifier_vs_option_value(new_option_widgets,new_options) # build dictionary to enable easy identifying/overwriting
        
        # update duplicated new widgets with existing ones
        for i,x in enumerate(old_options_list):
            loc = self._location_in_list(new_options_list,x)
            if loc != None:                                       # if old option exists in new list
                new_option_widgets[loc] = self._option_widgets[i] # replace new widget with existing one

        # update self.applied_settings to align with new widgets
        for k,v in self.applied_settings.items():
            if k in new_applied_settings:                         # if old option exists in new list
                new_applied_settings[k] = v                       # replace new applied_settings with existing ones

        # finalize
        self._option_widgets  = new_option_widgets                # new widgets
        self._options         = new_options                       # new option referential
        self.applied_settings = new_applied_settings              # new applied settings

        # pretty
        self._update_button_state()                               # update button state
        self._logging()                                           # show internals
        
    '''list-ify a dict'''
    def _dict2list(self,d : Dict[X,Y]) -> List[Tuple[X,Y]] :
        return [(k,v) for k,v in d.items()]
    
    '''make key-value keyed dico with values = widget.values '''
    def _option_identifier_vs_option_value(self,
        widgets : WIDGETS,
        options : OPTIONS,
        ) -> OPTIONID_VALUES :
        return {(k,v):w.value for w,(k,v) in zip(widgets,options.items())}
    
    '''locate position of x in l returning None if not found'''
    def _location_in_list(self,
        l : List[X],
        x : X
        ) -> int :
        try:
            return l.index(x) 
        except ValueError:
            return None
        
    def _logging(self) -> None :
        for k,v in self.applied_settings.items():
            if v:
                logging.debug(f'{self.name} : REFERENCE {k} = True')

In [ ]:
# __file__ exists if notebook called with %run but doesnt it called manually
# e.g. I only wish to run the example when calling this notebook directly
try:
    __file__
except NameError:
    # example
    a=CHECKBOXES()
    display(dir(a))
    display(a.widget)

In [ ]:
'''
a._options

new_options=a._populate_options(list('xyz'))
new_options

a.update_options(new_options)
'''
None